# Admin Panel

> Simple, composable admin utilities following FastHTML patterns

In [ ]:
#| default_exp admin

In [ ]:
#| export
from typing import Optional, Dict, Any, List, Union, Callable, Set, Type
from dataclasses import dataclass, field, fields
from datetime import datetime, date
from fasthtml.common import *
from fastlite import *
from monsterui.all import *
from launch_kit.permissions import require_role
from apswutils.db import NotFoundError
import math

In [ ]:
__all__ = ['setup_admin_routes', 'admin_list_view', 'admin_form', 'admin_table', 
           'get_field_input', 'clean_form_data']

## Quick Start

Following FastHTML patterns with simple, composable utilities:

```python
from launch_kit.admin import setup_admin_routes
from fasthtml.common import *

app, rt = fast_app()
db = Database('app.db')
products = db.create(Product, pk='id')

# Simple setup - just pass your table and router
setup_admin_routes(rt, 'products', products, Product)

# That's it! Routes are created at /admin/products/
```

## Core Functions

Simple, composable functions following FastHTML's functional approach:

In [ ]:
#| export
def setup_admin_routes(rt, # FastHTML router instance
                      name: str, # Model name (e.g., 'products')
                      table, # MiniDataAPI table
                      model_class: Type, # Dataclass model
                      path_prefix: str = '/admin', # URL prefix
                      auth_check: Optional[Callable] = None, # Auth function
                      per_page: int = 25 # Items per page
                      ):
    """Setup admin routes for a model using standard FastHTML patterns.
    
    This creates all CRUD routes following FastHTML conventions.
    """
    auth_check = auth_check or (lambda r, s: require_role('admin', r, s))
    base_path = f"{path_prefix}/{name}"
    
    # List view
    @rt(f"{base_path}/")
    def list_view(req, sess):
        if not auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        # Get query params
        page = int(req.query_params.get('page', 1))
        search = req.query_params.get('search', '')
        
        # Simple pagination
        offset = (page - 1) * per_page
        
        # Query with search
        if search:
            # Simple search across string fields
            items = _search_items(table, model_class, search, per_page, offset)
        else:
            items = table(limit=per_page, offset=offset, order_by='id DESC')
        
        total = len(table())
        total_pages = math.ceil(total / per_page)
        
        return admin_list_view(name, items, page, total_pages, search, base_path)
    
    # Create form (GET)
    @rt(f"{base_path}/new")
    def create_form(req, sess):
        if not auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        return admin_form(model_class, None, f"{base_path}/new", f"New {name.title()}")
    
    # Create handler (POST)
    @rt(f"{base_path}/new")
    async def create_handler(req, sess):
        if not auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        form_data = await req.form()
        try:
            cleaned = clean_form_data(model_class, dict(form_data))
            instance = model_class(**cleaned)
            table.insert(instance)
            return RedirectResponse(f"{base_path}/", status_code=303)
        except Exception as e:
            return admin_form(model_class, dict(form_data), f"{base_path}/new", 
                            f"New {name.title()}", error=str(e))
    
    # Edit form (GET)
    @rt(f"{base_path}/{{id}}/edit")
    def edit_form(req, sess, id: int):
        if not auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        try:
            item = table[id]
            item_dict = vars(item) if hasattr(item, '__dict__') else item
            return admin_form(model_class, item_dict, f"{base_path}/{id}/edit", 
                            f"Edit {name.title()}")
        except NotFoundError:
            return HTMLResponse("Item not found", status_code=404)
    
    # Edit handler (POST)
    @rt(f"{base_path}/{{id}}/edit")
    async def edit_handler(req, sess, id: int):
        if not auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        form_data = await req.form()
        try:
            item = table[id]
            item_dict = vars(item) if hasattr(item, '__dict__') else item
            
            cleaned = clean_form_data(model_class, dict(form_data))
            updated = {**item_dict, **cleaned}
            
            if 'updated_at' in model_class.__dataclass_fields__:
                updated['updated_at'] = datetime.utcnow()
            
            table.update(updated)
            return RedirectResponse(f"{base_path}/", status_code=303)
        except Exception as e:
            return admin_form(model_class, dict(form_data), f"{base_path}/{id}/edit", 
                            f"Edit {name.title()}", error=str(e))
    
    # Delete handler (POST)
    @rt(f"{base_path}/{{id}}/delete")
    def delete_handler(req, sess, id: int):
        if not auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        try:
            table.delete(id)
            if req.headers.get('HX-Request'):
                return ""
        except NotFoundError:
            pass
        
        return RedirectResponse(f"{base_path}/", status_code=303)

In [ ]:
#| export
def admin_list_view(name: str, # Model name
                   items: List, # List of items
                   page: int, # Current page
                   total_pages: int, # Total pages
                   search: str = '', # Search query
                   base_path: str = '' # Base URL path
                   ) -> FT:
    """Generate admin list view with table and pagination.
    
    Simple, reusable component following FastHTML patterns.
    """
    return Container(
        # Header with title and new button
        Div(
            H1(f"{name.title()} List", cls="text-3xl font-bold"),
            A(f"+ New {name.title()}", href=f"{base_path}/new", 
              cls="btn btn-primary"),
            cls="flex justify-between items-center mb-6"
        ),
        
        # Search form
        Form(
            Input(name="search", value=search, placeholder="Search...", cls="mr-2"),
            Button("Search", type="submit"),
            method="get",
            cls="mb-4"
        ),
        
        # Table
        admin_table(items, base_path),
        
        # Pagination
        _pagination(page, total_pages, base_path, search)
    )

In [ ]:
#| export
def admin_table(items: List, # List of items to display
               base_path: str = '' # Base URL path for actions
               ) -> FT:
    """Generate admin table component.
    
    Simple table component that works with any list of dict-like objects.
    """
    if not items:
        return Card(P("No items found", cls="text-center py-8"))
    
    # Get first item to determine columns
    first_item = items[0]
    if hasattr(first_item, '__dict__'):
        first_dict = vars(first_item)
    else:
        first_dict = first_item
    
    # Column names (exclude id from display)
    columns = [k for k in first_dict.keys() if k != 'id']
    
    # Headers
    headers = [Th(col.replace('_', ' ').title()) for col in columns]
    headers.append(Th("Actions", cls="text-right"))
    
    # Rows
    rows = []
    for item in items:
        item_dict = vars(item) if hasattr(item, '__dict__') else item
        
        # Data cells
        cells = []
        for col in columns:
            value = item_dict.get(col, '')
            # Format common types
            if isinstance(value, bool):
                value = "✓" if value else "✗"
            elif isinstance(value, datetime):
                value = value.strftime('%Y-%m-%d %H:%M')
            cells.append(Td(str(value)))
        
        # Action buttons
        actions = Td(
            A("Edit", href=f"{base_path}/{item_dict['id']}/edit",
              cls="text-primary hover:underline mr-3"),
            Button("Delete", 
                   hx_post=f"{base_path}/{item_dict['id']}/delete",
                   hx_confirm="Are you sure?",
                   hx_target="closest tr",
                   hx_swap="outerHTML",
                   cls="text-destructive hover:underline"),
            cls="text-right"
        )
        cells.append(actions)
        
        rows.append(Tr(*cells))
    
    return Card(
        Table(
            Thead(Tr(*headers)),
            Tbody(*rows),
            cls="w-full"
        )
    )

In [ ]:
#| export
def admin_form(model_class: Type, # Dataclass model
              data: Optional[Dict] = None, # Current data (for editing)
              action: str = '', # Form action URL
              title: str = 'Form', # Form title
              error: Optional[str] = None # Error message
              ) -> FT:
    """Generate admin form from dataclass.
    
    Simple form generator that creates appropriate inputs for each field type.
    """
    data = data or {}
    
    # Generate form fields
    form_fields = []
    for field in fields(model_class):
        if field.name == 'id':  # Skip primary key
            continue
        
        value = data.get(field.name, field.default if field.default != field.default_factory else '')
        input_component = get_field_input(field.name, field.type, value)
        form_fields.append(input_component)
    
    return Container(
        H1(title, cls="text-3xl font-bold mb-6"),
        Card(
            Form(
                Alert(error, variant="destructive", cls="mb-4") if error else None,
                *form_fields,
                Div(
                    Button("Save", type="submit", cls="btn btn-primary mr-2"),
                    A("Cancel", href="javascript:history.back()", 
                      cls="btn btn-outline"),
                    cls="mt-4"
                ),
                method="post",
                action=action,
                cls="space-y-4"
            ),
            cls="max-w-2xl"
        )
    )

In [ ]:
#| export
def get_field_input(name: str, # Field name
                   field_type: Type, # Field type
                   value: Any = '' # Current value
                   ) -> FT:
    """Get appropriate input component for field type.
    
    Simple function that maps Python types to HTML input types.
    """
    label = name.replace('_', ' ').title()
    
    # Handle Optional types
    if hasattr(field_type, '__origin__') and field_type.__origin__ is Union:
        args = field_type.__args__
        field_type = next((t for t in args if t != type(None)), str)
    
    # Boolean -> Checkbox
    if field_type is bool:
        return Div(
            Label(
                Input(type="checkbox", name=name, value="true", checked=bool(value)),
                label,
                cls="flex items-center space-x-2"
            )
        )
    
    # Numbers
    elif field_type in (int, float):
        return LabelInput(label, name=name, type="number", 
                         value=str(value) if value is not None else '',
                         step="0.01" if field_type is float else "1")
    
    # Date/DateTime
    elif field_type is datetime:
        if isinstance(value, datetime):
            value = value.strftime('%Y-%m-%dT%H:%M')
        return LabelInput(label, name=name, type="datetime-local", value=value or '')
    
    elif field_type is date:
        if isinstance(value, date):
            value = value.strftime('%Y-%m-%d')
        return LabelInput(label, name=name, type="date", value=value or '')
    
    # Large text fields
    elif any(word in name.lower() for word in ['description', 'content', 'body', 'notes']):
        return Div(
            Label(label, htmlFor=name),
            Textarea(value or '', name=name, id=name, rows=4, cls="textarea")
        )
    
    # Default to text input
    else:
        return LabelInput(label, name=name, type="text", value=value or '')

In [ ]:
#| export
def clean_form_data(model_class: Type, # Dataclass model
                   form_data: Dict # Raw form data
                   ) -> Dict:
    """Clean and convert form data to appropriate types.
    
    Simple type conversion from HTML form strings to Python types.
    """
    cleaned = {}
    
    for field in fields(model_class):
        if field.name == 'id':  # Skip primary key
            continue
            
        value = form_data.get(field.name)
        
        # Handle empty strings
        if value == '':
            if field.default != field.default_factory:
                value = field.default
            else:
                value = None
        
        # Type conversion
        if value is not None and value != '':
            field_type = field.type
            
            # Handle Optional types
            if hasattr(field_type, '__origin__') and field_type.__origin__ is Union:
                args = field_type.__args__
                field_type = next((t for t in args if t != type(None)), str)
            
            # Convert based on type
            if field_type is bool:
                value = value in ('true', 'on', '1', True)
            elif field_type is int:
                value = int(value)
            elif field_type is float:
                value = float(value)
            elif field_type is datetime:
                value = datetime.fromisoformat(value)
            elif field_type is date:
                value = datetime.strptime(value, '%Y-%m-%d').date()
        
        # Only include non-None values or required fields
        if value is not None or field.default == field.default_factory:
            cleaned[field.name] = value
    
    return cleaned

## Helper Functions

In [ ]:
#| export
def _search_items(table, model_class: Type, search: str, limit: int, offset: int):
    """Simple search across string fields."""
    # Get string fields from dataclass
    string_fields = [f.name for f in fields(model_class) 
                    if f.type is str and f.name != 'id']
    
    if not string_fields:
        return table(limit=limit, offset=offset)
    
    # Build search query
    search_conditions = ' OR '.join([f"{field} LIKE ?" for field in string_fields])
    search_args = [f"%{search}%" for _ in string_fields]
    
    return table(
        where=search_conditions,
        where_args=search_args,
        limit=limit,
        offset=offset,
        order_by='id DESC'
    )

def _pagination(page: int, total_pages: int, base_path: str, search: str = '') -> FT:
    """Simple pagination component."""
    if total_pages <= 1:
        return Div()
    
    links = []
    
    # Previous
    if page > 1:
        prev_query = f"?page={page-1}&search={search}" if search else f"?page={page-1}"
        links.append(A("← Previous", href=f"{base_path}/{prev_query}", cls="btn btn-outline mr-2"))
    
    # Pages
    for p in range(max(1, page-2), min(total_pages+1, page+3)):
        if p == page:
            links.append(Span(str(p), cls="btn btn-primary mr-2"))
        else:
            page_query = f"?page={p}&search={search}" if search else f"?page={p}"
            links.append(A(str(p), href=f"{base_path}/{page_query}", cls="btn btn-outline mr-2"))
    
    # Next
    if page < total_pages:
        next_query = f"?page={page+1}&search={search}" if search else f"?page={page+1}"
        links.append(A("Next →", href=f"{base_path}/{next_query}", cls="btn btn-outline"))
    
    return Div(*links, cls="mt-4 flex items-center")

## Usage Examples

### Simple Setup

In [ ]:
#| eval: false
from fasthtml.common import *
from launch_kit.admin import setup_admin_routes
from dataclasses import dataclass
from typing import Optional

# Standard FastHTML setup
app, rt = fast_app()
db = Database('app.db')

@dataclass
class Product:
    name: str
    price: float
    category: str
    is_active: bool = True
    id: Optional[int] = None

products = db.create(Product, pk='id')

# One line setup
setup_admin_routes(rt, 'products', products, Product)

# That's it! Routes created:
# /admin/products/ - list view
# /admin/products/new - create form
# /admin/products/{id}/edit - edit form
# /admin/products/{id}/delete - delete handler

### Custom Components

In [ ]:
#| eval: false
# Use individual components for custom layouts
from launch_kit.admin import admin_table, admin_form, get_field_input

@rt('/custom-admin')
def custom_admin(req, sess):
    products = products_table()  # Your table
    
    return Container(
        H1("Custom Admin Dashboard"),
        admin_table(products, '/admin/products')
    )

@rt('/custom-form')
def custom_form(req, sess):
    return admin_form(Product, action='/save-product', title='Add Product')

In [ ]:
#| eval: false
# Setup admin for multiple models
@dataclass
class Category:
    name: str
    description: str = ""
    id: Optional[int] = None

@dataclass
class Order:
    customer_email: str
    total: float
    status: str = "pending"
    created_at: datetime = field(default_factory=datetime.utcnow)
    id: Optional[int] = None

# Create tables
categories = db.create(Category, pk='id')
orders = db.create(Order, pk='id')

# Setup admin routes for each
setup_admin_routes(rt, 'products', products, Product)
setup_admin_routes(rt, 'categories', categories, Category)
setup_admin_routes(rt, 'orders', orders, Order)

# Create admin dashboard
@rt('/admin')
def admin_dashboard(req, sess):
    return Container(
        H1("Admin Dashboard", cls="text-3xl font-bold mb-6"),
        Grid(
            Card(H3("Products"), A("Manage →", href="/admin/products/")),
            Card(H3("Categories"), A("Manage →", href="/admin/categories/")),
            Card(H3("Orders"), A("Manage →", href="/admin/orders/")),
            cols=3
        )
    )

## Tests

Let's test all the admin functionality to ensure it works correctly:

### Test Setup

In [ ]:
# Create test database and models for testing
from tempfile import NamedTemporaryFile
import os

# Create a temporary database for testing
temp_db = NamedTemporaryFile(delete=False, suffix='.db')
test_db_path = temp_db.name
temp_db.close()

# Initialize test database
test_db = Database(test_db_path)

# Define test models
@dataclass
class TestProduct:
    name: str
    price: float
    description: str = ""
    is_active: bool = True
    created_at: datetime = field(default_factory=datetime.utcnow)
    id: Optional[int] = None

@dataclass  
class TestUser:
    username: str
    email: str
    role: str = "user"
    is_active: bool = True
    id: Optional[int] = None

# Create tables
test_products = test_db.create(TestProduct, pk='id')
test_users = test_db.create(TestUser, pk='id')

# Insert test data
test_products.insert(TestProduct(name="Widget", price=19.99, description="A useful widget"))
test_products.insert(TestProduct(name="Gadget", price=29.99, description="An amazing gadget", is_active=False))
test_products.insert(TestProduct(name="Doohickey", price=9.99))

test_users.insert(TestUser(username="alice", email="alice@example.com", role="admin"))
test_users.insert(TestUser(username="bob", email="bob@example.com"))

print(f"Test database created with {len(test_products())} products and {len(test_users())} users")

Test database created with 3 products and 2 users


<string>:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).


### Test Form Data Cleaning

In [ ]:
# Test clean_form_data function
test_form_data = {
    'name': 'Test Product',
    'price': '49.99',
    'description': 'A test product',
    'is_active': 'true',
    'created_at': '2024-01-01T10:00'
}

cleaned = clean_form_data(TestProduct, test_form_data)

# Test type conversions
assert cleaned['name'] == 'Test Product'
assert cleaned['price'] == 49.99
assert isinstance(cleaned['price'], float)
assert cleaned['is_active'] == True
assert isinstance(cleaned['is_active'], bool)
assert isinstance(cleaned['created_at'], datetime)

# Test empty values
test_form_data_empty = {
    'name': 'Product',
    'price': '10.0',
    'description': '',  # Empty string should become None or default
    'is_active': ''    # Empty string for bool
}

cleaned_empty = clean_form_data(TestProduct, test_form_data_empty)
assert cleaned_empty['description'] == ""  # Default value
assert 'is_active' not in cleaned_empty or cleaned_empty['is_active'] == True  # Default value

print("✓ Form data cleaning tests passed")

✓ Form data cleaning tests passed


### Test Field Input Generation

In [ ]:
# Test get_field_input for different field types
from typing import Optional

# Test text input
text_input = get_field_input('name', str, 'Test Name')
assert isinstance(text_input, FT)
assert 'Test Name' in str(text_input)

# Test number input
number_input = get_field_input('price', float, 19.99)
assert isinstance(number_input, FT)
assert '19.99' in str(number_input)
assert 'type="number"' in str(number_input)

# Test boolean checkbox
bool_input = get_field_input('is_active', bool, True)
assert isinstance(bool_input, FT)
assert 'checkbox' in str(bool_input)
assert 'checked' in str(bool_input)

# Test datetime input
dt = datetime(2024, 1, 1, 10, 30)
datetime_input = get_field_input('created_at', datetime, dt)
assert isinstance(datetime_input, FT)
assert 'datetime-local' in str(datetime_input)
assert '2024-01-01T10:30' in str(datetime_input)

# Test textarea for description fields
textarea_input = get_field_input('description', str, 'Long text')
assert isinstance(textarea_input, FT)
assert 'textarea' in str(textarea_input).lower()
assert 'Long text' in str(textarea_input)

# Test Optional types
optional_input = get_field_input('optional_field', Optional[str], 'Value')
assert isinstance(optional_input, FT)

print("✓ Field input generation tests passed")

✓ Field input generation tests passed


### Test Admin Table Generation

In [ ]:
# Test admin_table function
products = test_products()
table_html = admin_table(products, '/admin/products')

# Convert to string for testing
table_str = str(table_html)

# Check table structure
assert 'table' in table_str.lower()
assert 'thead' in table_str.lower()
assert 'tbody' in table_str.lower()

# Check headers are present (excluding id)
assert 'Name' in table_str
assert 'Price' in table_str
assert 'Description' in table_str
assert 'Is Active' in table_str
assert 'Actions' in table_str

# Check data is present
assert 'Widget' in table_str
assert '19.99' in table_str
assert 'Gadget' in table_str

# Check action buttons
assert '/admin/products/1/edit' in table_str
assert '/admin/products/1/delete' in table_str
assert 'hx-post' in table_str
assert 'hx-confirm' in table_str

# Test empty table
empty_table = admin_table([], '/admin/products')
empty_str = str(empty_table)
assert 'No items found' in empty_str

print("✓ Admin table generation tests passed")

✓ Admin table generation tests passed


### Test Admin Form Generation

In [ ]:
# Test admin_form for create (no data)
create_form = admin_form(TestProduct, None, '/admin/products/new', 'New Product')
create_str = str(create_form)

# Check form structure
assert 'form' in create_str.lower()
assert 'method="post"' in create_str
assert 'action="/admin/products/new"' in create_str
assert 'New Product' in create_str

# Check fields are present (excluding id)
assert 'name="name"' in create_str
assert 'name="price"' in create_str
assert 'name="description"' in create_str
assert 'name="is_active"' in create_str

# Check buttons
assert 'Save' in create_str
assert 'Cancel' in create_str

# Test admin_form for edit (with data)
product_data = {
    'id': 1,
    'name': 'Test Widget',
    'price': 29.99,
    'description': 'A test widget',
    'is_active': True,
    'created_at': datetime.now()
}
edit_form = admin_form(TestProduct, product_data, '/admin/products/1/edit', 'Edit Product')
edit_str = str(edit_form)

# Check pre-filled values
assert 'Test Widget' in edit_str
assert '29.99' in edit_str
assert 'A test widget' in edit_str
assert 'Edit Product' in edit_str

# Test form with error
error_form = admin_form(TestProduct, None, '/admin/products/new', 'New Product', 
                       error='Name is required')
error_str = str(error_form)
assert 'Name is required' in error_str
assert 'alert' in error_str.lower()

print("✓ Admin form generation tests passed")

✓ Admin form generation tests passed


### Test Admin List View

In [ ]:
# Test admin_list_view
products = test_products()
list_view = admin_list_view('products', products, page=1, total_pages=1, 
                            search='', base_path='/admin/products')
list_str = str(list_view)

# Check main components
assert 'Products List' in list_str
assert '+ New Products' in list_str
assert '/admin/products/new' in list_str

# Check search form
assert 'search' in list_str.lower()
assert 'placeholder="Search..."' in list_str

# Check table is included
assert 'Widget' in list_str
assert 'Gadget' in list_str

# Test with search
search_view = admin_list_view('products', products, page=1, total_pages=1,
                             search='widget', base_path='/admin/products')
search_str = str(search_view)
assert 'value="widget"' in search_str

# Test with pagination
paginated_view = admin_list_view('products', products, page=2, total_pages=3,
                                search='', base_path='/admin/products')
paginated_str = str(paginated_view)
assert '← Previous' in paginated_str
assert 'Next →' in paginated_str
assert 'page=1' in paginated_str
assert 'page=3' in paginated_str

print("✓ Admin list view tests passed")

TypeError: functools.partial(<function ft_hx at 0x10dfedc60>, 'main') got multiple values for keyword argument 'cls'

### Test Search Functionality

In [ ]:
# Test _search_items helper
# Search for "widget" should find the Widget product (case-insensitive)
search_results = _search_items(test_products, TestProduct, 'widget', limit=10, offset=0)
assert len(search_results) >= 1
# Check if Widget is in results (handle both dict and object)
found_widget = False
for p in search_results:
    name = p.name if hasattr(p, 'name') else p.get('name', '')
    if 'widget' in name.lower():
        found_widget = True
        break
assert found_widget, "Should find Widget product"

# Search for "amazing" should find the Gadget (in description)
search_results = _search_items(test_products, TestProduct, 'amazing', limit=10, offset=0)
assert len(search_results) >= 1

# Test pagination in search
search_results = _search_items(test_products, TestProduct, '', limit=2, offset=1)
assert len(search_results) <= 2  # Should get at most 2 items

# Test model with no string fields
@dataclass
class NumericModel:
    count: int
    value: float
    id: Optional[int] = None

numeric_table = test_db.create(NumericModel, pk='id')
numeric_table.insert(NumericModel(count=1, value=1.5))

# Should return results even with no string fields
results = _search_items(numeric_table, NumericModel, 'test', limit=10, offset=0)
assert isinstance(results, list)  # Should not error and return a list

print("✓ Search functionality tests passed")

### Test Pagination

In [ ]:
# Test _pagination helper
# Test no pagination needed (1 page)
pagination = _pagination(1, 1, '/admin/products')
assert str(pagination) == '<div></div>'  # Empty div when no pagination needed

# Test basic pagination
pagination = _pagination(2, 5, '/admin/products')
pag_str = str(pagination)
assert '← Previous' in pag_str
assert 'Next →' in pag_str
assert 'page=1' in pag_str
assert 'page=3' in pag_str
assert '<span' in pag_str  # Current page as span

# Test first page
pagination = _pagination(1, 3, '/admin/products')
pag_str = str(pagination)
assert '← Previous' not in pag_str  # No previous on first page
assert 'Next →' in pag_str
assert 'page=2' in pag_str

# Test last page
pagination = _pagination(5, 5, '/admin/products')
pag_str = str(pagination)
assert '← Previous' in pag_str
assert 'Next →' not in pag_str  # No next on last page
assert 'page=4' in pag_str

# Test with search parameter
pagination = _pagination(2, 3, '/admin/products', search='widget')
pag_str = str(pagination)
assert 'search=widget' in pag_str
# Check both parameters are present (order doesn't matter)
assert 'page=1' in pag_str and 'search=widget' in pag_str

print("✓ Pagination tests passed")

### Test Route Setup (Mock)

In [ ]:
# Test setup_admin_routes with a mock router
class MockRouter:
    """Mock router to test route registration"""
    def __init__(self):
        self.routes = {}
    
    def __call__(self, path):
        def decorator(func):
            # Store both GET and POST handlers
            if path not in self.routes:
                self.routes[path] = {}
            
            # Determine method based on function name
            if 'handler' in func.__name__:
                self.routes[path]['POST'] = func
            else:
                self.routes[path]['GET'] = func
            return func
        return decorator

# Create mock router and setup routes
mock_rt = MockRouter()
setup_admin_routes(mock_rt, 'products', test_products, TestProduct)

# Check all expected routes were created
expected_routes = [
    '/admin/products/',
    '/admin/products/new',
    '/admin/products/{id}/edit',
    '/admin/products/{id}/delete'
]

for route in expected_routes:
    assert route in mock_rt.routes, f"Route {route} not found"

# Check route has both GET and POST where appropriate
assert 'GET' in mock_rt.routes['/admin/products/new']
assert 'POST' in mock_rt.routes['/admin/products/new']
assert 'GET' in mock_rt.routes['/admin/products/{id}/edit']
assert 'POST' in mock_rt.routes['/admin/products/{id}/edit']

# Test custom path prefix
mock_rt2 = MockRouter()
setup_admin_routes(mock_rt2, 'users', test_users, TestUser, path_prefix='/backend')

assert '/backend/users/' in mock_rt2.routes
assert '/admin/users/' not in mock_rt2.routes

print("✓ Route setup tests passed")

### Test Edge Cases

In [ ]:
# Test edge cases

# 1. Model with all optional fields
@dataclass
class OptionalModel:
    name: Optional[str] = None
    count: Optional[int] = None
    active: Optional[bool] = None
    id: Optional[int] = None

# Should handle optional fields gracefully
form = admin_form(OptionalModel, None, '/test', 'Test')
assert isinstance(form, FT)

# 2. Model with complex types
from typing import List
@dataclass 
class ComplexModel:
    tags: List[str] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)
    id: Optional[int] = None

# Should handle complex types (defaulting to text inputs)
complex_form = admin_form(ComplexModel, None, '/test', 'Test')
assert isinstance(complex_form, FT)

# 3. Empty form data cleaning
empty_data = {}
cleaned = clean_form_data(TestProduct, empty_data)
# Should handle empty data without errors
assert isinstance(cleaned, dict)

# 4. Special characters in data
special_data = {
    'name': 'Product & <Special>',
    'price': '19.99',
    'description': 'Contains "quotes" and \'apostrophes\''
}
cleaned = clean_form_data(TestProduct, special_data)
assert cleaned['name'] == 'Product & <Special>'
assert cleaned['description'] == 'Contains "quotes" and \'apostrophes\''

# 5. Very long field names
@dataclass
class LongFieldModel:
    this_is_a_very_long_field_name_that_should_still_work: str = ""
    id: Optional[int] = None

input_elem = get_field_input('this_is_a_very_long_field_name_that_should_still_work', str, 'test')
assert isinstance(input_elem, FT)
assert 'This Is A Very Long Field Name That Should Still Work' in str(input_elem)

print("✓ Edge case tests passed")

### Cleanup

### Integration Test Example

In [ ]:
# Full integration test showing how all components work together
from dataclasses import dataclass
from typing import Optional
from datetime import datetime

# Define a complete model
@dataclass
class Article:
    title: str
    content: str
    author: str
    published: bool = False
    views: int = 0
    published_at: Optional[datetime] = None
    id: Optional[int] = None

# Create test database
integration_db = Database(':memory:')
articles = integration_db.create(Article, pk='id')

# Insert test data
articles.insert(Article(
    title="Getting Started with FastHTML",
    content="FastHTML is a modern web framework...",
    author="John Doe",
    published=True,
    views=150,
    published_at=datetime.now()
))
articles.insert(Article(
    title="Building Admin Panels",
    content="Admin panels are essential for managing content...",
    author="Jane Smith",
    published=False,
    views=0
))

# Test all components work together
# 1. List view
list_html = admin_list_view('articles', articles(), 1, 1, '', '/admin/articles')
assert 'Getting Started with FastHTML' in str(list_html)
assert '150' in str(list_html)  # views

# 2. Create form
create_form_html = admin_form(Article, None, '/admin/articles/new', 'New Article')
assert 'name="title"' in str(create_form_html)
assert 'name="content"' in str(create_form_html)
assert 'name="published"' in str(create_form_html)

# 3. Edit form with data
article = articles[1]
article_dict = vars(article) if hasattr(article, '__dict__') else article
edit_form_html = admin_form(Article, article_dict, f'/admin/articles/1/edit', 'Edit Article')
assert 'Getting Started with FastHTML' in str(edit_form_html)

# 4. Test form data processing
form_data = {
    'title': 'New Article',
    'content': 'Content here',
    'author': 'Test Author',
    'published': 'true',
    'views': '100',
    'published_at': '2024-01-01T10:00'
}
cleaned = clean_form_data(Article, form_data)
assert cleaned['title'] == 'New Article'
assert cleaned['published'] == True
assert cleaned['views'] == 100
assert isinstance(cleaned['published_at'], datetime)

print("✓ Integration test passed - all components work together!")

In [ ]:
# Clean up test database
os.unlink(test_db_path)
print("✓ Test database cleaned up")

print("\n🎉 All admin tests passed successfully!")

## Summary

This improved admin module follows FastHTML and Answer.ai best practices:

### ✅ What's Better:

1. **Uses `rt` Router Pattern** - Routes registered on the FastHTML router, not app instance
2. **Functional Approach** - Simple functions instead of complex classes
3. **Composable** - Each function has a single purpose and can be used independently
4. **Simple Setup** - One line to add admin: `setup_admin_routes(rt, 'products', products, Product)`
5. **Transparent** - No hidden magic, every component is inspectable
6. **Progressive Enhancement** - Start simple, customize as needed
7. **Standard Patterns** - Uses FastHTML's `@rt`, `req`, `sess`, and form handling

### 🎯 Key Improvements:

- **Removed Complex Class**: No more `AdminPanel` class with dozens of methods
- **Follows `rt` Pattern**: Routes use `@rt()` decorator like all FastHTML apps
- **Single Purpose Functions**: Each function does one thing well
- **Easy to Understand**: Linear flow from simple to complex use cases
- **Customizable Components**: Use `admin_table()`, `admin_form()` in your own routes
- **FastHTML Native**: Uses standard req/sess/form patterns throughout

This approach embodies Jeremy Howard's philosophy: "Simple things should be simple, complex things should be possible." You can set up admin in one line, but every component is accessible for customization.